In [1]:
from eventregistry import *
import json, os, sys

from dotenv import load_dotenv
import os
from eventregistry import EventRegistry


load_dotenv()  # This loads the environment variables from .env file
api_key = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

In [6]:
# generate a query for the concept "Tesla Cybertruck" in Chinese
conceptUri = "Russo-Ukrainian War"
lang = "eng"
q = QueryArticlesIter(conceptUri=er.getConceptUri(conceptUri), lang=lang)

filename = conceptUri + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=100,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [3]:
from openai import OpenAI

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def get_summary(
    art,
    client,
    model="gpt-3.5-turbo",
    language="English",
):

    prompt_complete = f"Summarize the following text in {language}: " + art["body"]
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [4]:
# summarize the articles in both English and Chinese
conceptUri = "Tesla Cybertruck"
lang = "zho"

src_filename = conceptUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in articles:
        src_art["summary_3.5_eng"] = get_summary(
            src_art, client, language="English", model="gpt-3.5-turbo"
        )
        src_art["summary_3.5_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-3.5-turbo"
        )
        src_art["summary_4o_eng"] = get_summary(
            src_art, client, language="English", model="gpt-4o"
        )
        src_art["summary_4o_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-4o"
        )
        des_articles.append(src_art)
        print(f'Summary in English(3.5): {src_art["summary_3.5_eng"]}')
        print(f'Summary in English(4o): {src_art["summary_4o_eng"]}')
        print(f'Summary in Chinese(3.5): {src_art["summary_3.5_zho"]}')
        print(f'Summary in Chinese(4o): {src_art["summary_4o_zho"]}')
        print(f'Original text: {src_art["body"]}')
        print("_____________________________")

des_filename = "summary" + "_" + conceptUri + "_" + lang + ".json"
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

Summary in English(3.5): According to the revised Article 6, Paragraph 3 of the "Computer Network Content Rating Processing Measures," websites are required to display ratings on their homepage or restricted pages in accordance with the regulations of the Taiwan Website Classification Promotion Foundation. The Taiwan Website Classification Promotion Foundation (TICRF) website is http://www.ticrf.org.tw. In entertainment news, popular YouTuber MrBeast celebrated his 26th birthday by giving away 26 Tesla cars, including 25 Model 3 cars and one Cybertruck. Participants can enter by liking and commenting on MrBeast's Instagram post and tagging two friends, with the giveaway limited to certain countries excluding Taiwan.
Summary in English(4o): According to the revised Article 6, Clause 3 of the "Regulations for the Classification of Computer Network Content," websites must display classification labels on their homepage or restricted pages as per the Taiwan Internet Content Rating Promotio

In [5]:
conceptUri = "Tesla Cybertruck"
lang = "eng"
q = QueryArticlesIter(conceptUri=er.getConceptUri(conceptUri), lang=lang)

filename = conceptUri + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [6]:
src_filename = conceptUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in articles:
        src_art["summary_3.5_eng"] = get_summary(
            src_art, client, language="English", model="gpt-3.5-turbo"
        )
        src_art["summary_4o_eng"] = get_summary(
            src_art, client, language="English", model="gpt-4o"
        )

        des_articles.append(src_art)
        print(f'Summary in English(3.5): {src_art["summary_3.5_eng"]}')
        print(f'Summary in English(4o): {src_art["summary_4o_eng"]}')

        print(f'Original text: {src_art["body"]}')
        print("_____________________________")

des_filename = "summary" + "_" + conceptUri + "_" + lang + ".json"
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

Summary in English(3.5): Tesla is hinting at innovative advancements in autonomous driving and robotaxis, with a major unveiling set for August. A new camera setup was spotted on the prototype model of Model 3, sparking intrigue. Tesla's future driverless vehicles will utilize next-generation architecture and manufacturing processes, but these innovations may not be present in the upcoming robotaxi reveal. The automaker suggests blending current model characteristics with next-gen features until then. The prototype models spotted in Palo Alto may give clues about the final product, with unusual camera placements hinting at experimental setups for autonomous driving features. Tesla's camera-based approach, without LiDAR sensors, indicates confidence in their technology. The company's innovative approach keeps the automotive world guessing and ensures an exciting and unpredictable future of transportation.
Summary in English(4o): Tesla is gearing up for a major reveal in August, focusing

In [5]:
keywords = "俄乌战争"
lang = "zho"
q = QueryArticlesIter(keywords=keywords, lang=lang)

filename = keywords + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=100,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [8]:
# summarize the articles in both English and Chinese
keywords = "GPT-4o"
lang = "zho"

src_filename = keywords + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in articles:
        src_art["summary_3.5_eng"] = get_summary(
            src_art, client, language="English", model="gpt-3.5-turbo"
        )
        src_art["summary_3.5_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-3.5-turbo"
        )
        src_art["summary_4o_eng"] = get_summary(
            src_art, client, language="English", model="gpt-4o"
        )
        src_art["summary_4o_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-4o"
        )
        des_articles.append(src_art)
        print(f'Summary in English(3.5): {src_art["summary_3.5_eng"]}')
        print(f'Summary in English(4o): {src_art["summary_4o_eng"]}')
        print(f'Summary in Chinese(3.5): {src_art["summary_3.5_zho"]}')
        print(f'Summary in Chinese(4o): {src_art["summary_4o_zho"]}')
        print(f'Original text: {src_art["body"]}')
        print("_____________________________")

des_filename = "summary" + "_" + keywords + "_" + lang + ".json"
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

Summary in English(3.5): With the release of new generation large models by OpenAI and Google, the competition in generative artificial intelligence (AI) has reached a new peak. Google introduced the latest multimodal AI products at the I/O developer conference, including the universal AI assistant Project Astra powered by the upgraded Gemini model and the Veo video model targeting Sora.

Project Astra can analyze and respond to voice commands based on what it sees through the camera, while also engaging in real-time voice conversations with humans. Veo has the ability to generate 1080p high-definition videos based on prompts provided in various formats such as text, images, and videos. Google CEO Sundar Pichai stated that all of Google's work revolves around the generative AI model Gemini.

The competition between large tech companies has intensified, with OpenAI releasing the latest AI model GPT-4o just before Google's I/O conference. GPT-4o is the latest version of OpenAI's chatbot,

In [9]:
keywords = "GPT-4o"
lang = "eng"
q = QueryArticlesIter(
    keywords=keywords,
    lang=lang,
)

filename = keywords + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [11]:
# summarize the articles in both English
keywords = "GPT-4o"
lang = "eng"

src_filename = keywords + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in articles:
        src_art["summary_3.5_eng"] = get_summary(
            src_art, client, language="English", model="gpt-3.5-turbo"
        )
        src_art["summary_4o_eng"] = get_summary(
            src_art, client, language="English", model="gpt-4o"
        )
        des_articles.append(src_art)
        print(f'Summary in English(3.5): {src_art["summary_3.5_eng"]}')
        print(f'Summary in English(4o): {src_art["summary_4o_eng"]}')
        print(f'Original text: {src_art["body"]}')
        print("_____________________________")

des_filename = "summary" + "_" + keywords + "_" + lang + ".json"
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

Summary in English(3.5): Tech companies, including Google and Microsoft, are heavily investing in AI technology. Microsoft's upcoming Build 2024 conference will focus on AI advancements, new Surface hardware, and Windows 11 updates. The event is expected to showcase new consumer PCs with improved AI performance thanks to the Snapdragon X Elite chip. Microsoft's AI Explorer will introduce machine learning-based features like a revamped search tool and contextual suggestions. The company's Copilot tools, including Github Copilot and Bing Chat, are being integrated into various products. Microsoft is also working on making Copilot more local to improve response times. The conference will offer sessions on customizing Copilot and integrating it into various Microsoft apps. Other manufacturers are also expected to release AI-powered PCs with Qualcomm's Snapdragon X Elite Chip. Microsoft, like other tech giants, is heavily relying on AI to enhance the capabilities of its products.
Summary in

In [12]:
q = QueryEventsIter(conceptUri=er.getConceptUri("OpenAI"))
for event in q.execQuery(er, sortBy="size", maxItems=1):
    print(json.dumps(event, indent=4))

{
    "uri": "eng-9564068",
    "concepts": [
        {
            "uri": "http://en.wikipedia.org/wiki/OpenAI",
            "type": "wiki",
            "score": 100,
            "label": {
                "eng": "OpenAI"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/ChatGPT",
            "type": "wiki",
            "score": 86,
            "label": {
                "eng": "ChatGPT"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Artificial_intelligence",
            "type": "wiki",
            "score": 80,
            "label": {
                "eng": "Artificial intelligence"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Google",
            "type": "org",
            "score": 29,
            "label": {
                "eng": "Google"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Model",
            "type": "wiki",
            "score"

In [13]:
print(event["summary"]["zho"])

聊天機器人ChatGPT開發公司OpenAI，搶在Google I/O 2024開發者大會之前，5月13日宣布推出新AI模型「GPT-4o」，能夠進行十分逼真的人機對話，即時回應之外，還能融合文字與影像互動，在相關領域堪稱前所未見，此功能將免費提供，有望再次提升ChatGPT相關領域的領導地位。展示活動上，以溫暖女聲呈現的聊天機器人與研究人員流利對話、語氣自然。

OpenAI研究人員表示，「GPT-4o」的特


In [14]:
print(event["summary"]["eng"])

OpenAI's newest Artificial Intelligence model, GPT-4o, brings significant advancements to ChatGPT, improving its capabilities to understand images, text, and multiple languages more efficiently.

After the announcement of ChatGPT 4o, searches on Google on how to use this Artificial Intelligence model increased.

The rollout of this cutting-edge technology will occur gradually, ensuring that all users can access its powerful features seamlessly.

The GPT-4o model will be available to all ChatGPT u


In [15]:
q = QueryArticlesIter(
    sourceUri=er.getSourceUri("news.sina.com.cn"),
)

print("Number of results: %d" % q.count(er))

for art in q.execQuery(er, sortBy="rel", maxItems=2):
    print(json.dumps(art, indent=4))

Number of results: 1315
{
    "uri": "2024-05-358352422",
    "lang": "zho",
    "isDuplicate": true,
    "date": "2024-05-16",
    "time": "16:10:22",
    "dateTime": "2024-05-16T16:10:22Z",
    "dateTimePub": "2024-05-16T15:51:01Z",
    "dataType": "news",
    "sim": 0,
    "url": "https://news.sina.com.cn/c/2024-05-16/doc-inavnaav9413145.shtml",
    "title": "\u4fc4\u5a92\uff1a\u666e\u4eac\u5728\u534e\u8868\u793a\uff0c\u5728\u4e2d\u56fd\u611f\u89c9\u5c31\u50cf\u5728\u5bb6\u91cc\u4e00\u6837\uff0c\u4e2d\u56fd\u6587\u5316\u5728\u4fc4\u7f57\u65af\u4e5f\u8d8a\u6765\u8d8a\u53d7\u6b22\u8fce",
    "body": "[\u73af\u7403\u7f51\u62a5\u9053]5\u670816\u65e5\u4e0b\u5348\uff0c\u4e2d\u56fd\u56fd\u5bb6\u4e3b\u5e2d\u4e60\u8fd1\u5e73\u548c\u4fc4\u7f57\u65af\u603b\u7edf\u666e\u4eac\u5728\u5317\u4eac\u56fd\u5bb6\u5927\u5267\u9662\u5171\u540c\u51fa\u5e2d\"\u4e2d\u4fc4\u6587\u5316\u5e74\"\u5f00\u5e55\u5f0f\u66a8\u5e86\u795d\u4e2d\u4fc4\u5efa\u4ea475\u5468\u5e74\u4e13\u573a\u97f3\u4e50\u4f1a\u3002\n\n\u7e

In [16]:
print(art["body"])

@平安_松滋 微博5月16日发布情况通报。内容如下：

近日，网民视频反映我局警车在沪渝高速拦停一辆私家车。省市县公安机关派出督察、纪检监察部门开展调查。经查，5月10日下午，我局民警岑某、辅警罗某驾驶警车执行勤务时，超车变道拦停私家车。5月11日下午，涉事民警辅警被停止执行职务，目前正接受进一步调查。我局将根据调查结果，依法依纪严肃处理。我局将深刻吸取教训，诚恳接受社会监督，进一步加强队伍教育管理。

相关报道

湖北松滋一警车高速上逼停小车，当地最新回应：两民警已停职，正讨论怎么处理


In [17]:
keywords = "Microsoft Azure AI"
lang = "eng"
q = QueryArticlesIter(
    keywords=keywords,
    lang=lang,
)

filename = keywords + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True, ranking=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [18]:
q = QueryEventsIter(keywords=("Microsoft Azure AI"))
for event in q.execQuery(er, sortBy="size", maxItems=2):
    print(json.dumps(event, indent=4))

{
    "uri": "eng-9541338",
    "concepts": [
        {
            "uri": "http://en.wikipedia.org/wiki/Artificial_intelligence",
            "type": "wiki",
            "score": 100,
            "label": {
                "eng": "Artificial intelligence"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Generative_artificial_intelligence",
            "type": "wiki",
            "score": 28,
            "label": {
                "eng": "Generative artificial intelligence"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Microsoft",
            "type": "org",
            "score": 15,
            "label": {
                "eng": "Microsoft"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/ChatGPT",
            "type": "wiki",
            "score": 14,
            "label": {
                "eng": "ChatGPT"
            }
        },
        {
            "uri": "http://en.wikipedia.or

In [20]:
keywords = "GPT-4o"
q = QueryEventsIter(keywords=(keywords))

filename = keywords + "_" + ".json"
if os.path.exists(filename):
    os.remove(filename)

events = []
for event in q.execQuery(er, sortBy="size", maxItems=2):
    events.append(event)

with open(filename, "w") as f:
    json.dump(events, f, indent=4)

In [21]:
keywords = "OpenAI"
q = QueryEventsIter(keywords=(keywords))

filename = keywords + "_events" + ".json"
if os.path.exists(filename):
    os.remove(filename)

events = []
for event in q.execQuery(er, sortBy="size", maxItems=2):
    events.append(event)

with open(filename, "w") as f:
    json.dump(events, f, indent=4)